In [1]:
from Binaries import *
Th1 = HardwareClassifier(["th1"])
ToT = HardwareClassifier(["tot"])
ToTd = HardwareClassifier(["totd"])

In [2]:
vem_compatibility = np.array([163.235 for _ in range(3)])

EventsLTP = EventGenerator("/cr/tempdata01/filip/QGSJET-II/LTP/extracted/", apply_downsampling = True, q_peak = vem_compatibility, split = 1, real_background = False)

In [ ]:
EventsLTP.__getitem__(0)[0].__plot__()

In [3]:
Th1(EventsLTP.__getitem__(0)[0])

True

In [4]:
Th1(EventsLTP.__getitem__(0)[-1])

[]


False

In [5]:
ToT(EventsLTP.__getitem__(0)[1])

[<function HardwareClassifier.ToT at 0x7f4f5c79ee50>]


True